# **Problem Statement**  
## **22. Find articulation points in a graph.**

Given an undirected graph, identify all articulation points (cut vertices).
A vertex is an articulation point if removing it (and its edges) increases the number of connected components in the graph.

### Constraints & Example Inputs/Outputs

#### Constraints
- Graph can be:
    - Connected or disconnected
    - Sparse or dense
- Number of vertices: 1 ≤ V ≤ 10^5
- Number of edges: 0 ≤ E ≤ 2 × 10^5
- Graph is undirected

Example Input:
```python
V = 5
Edges:
0 -- 1
1 -- 2
2 -- 0
1 -- 3
3 -- 4
```

Expected Output
```python
Articulation Points: [1, 3]
```


### Solution Approach

Here are the 2 best possible approaches:

#### 1. Brute Force Approach (Naive)
a. For every vertex v:
- Remove v and its edges
- Count the number of connected components using DFS/BFS
- Restore v
- If component count increases → v is an articulation point
→ Time Complexity: O(V * (V + E)) (too slow for large graphs)

#### 2. Optimized Approach (Tarjan’s Algorithm)
Use DFS to compute:
- disc[v]: Discovery time of vertex v
- low[v]: The earliest discovered vertex reachable from v via back-edges

Articulation point conditions:
1. Root of DFS tree with two or more children
2. Non-root vertex v where:
    - low[child] ≥ disc[v]
  
This approach visits each node and edge only once.

→ Time Complexity: O(V + E)
→ Space: O(V)

### Solution Code

In [1]:
# Approach 1: Brute Force Implementation
from collections import defaultdict, deque

def bfs_components(graph, removed, V):
    visited = [False] * V
    count = 0
    
    for start in range(V):
        if start == removed or visited[start]:
            continue
        
        queue = deque([start])
        visited[start] = True
        count += 1
        
        while queue:
            node = queue.popleft()
            for nei in graph[node]:
                if nei != removed and not visited[nei]:
                    visited[nei] = True
                    queue.append(nei)
    
    return count


def articulation_points_bruteforce(V, edges):
    # Build graph
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
    
    original_components = bfs_components(graph, removed=-1, V=V)
    articulation_points = []
    
    for v in range(V):
        components = bfs_components(graph, removed=v, V=V)
        if components > original_components:
            articulation_points.append(v)
    
    return articulation_points


# Example test
V = 5
edges = [(0,1), (1,2), (2,0), (1,3), (3,4)]
print("Brute Force APs:", articulation_points_bruteforce(V, edges))


Brute Force APs: [1, 3]


### Alternative Solution

In [2]:
# Approach 2: Optimized Implementation (Tarjan’s Algorithm)
from collections import defaultdict

def articulation_points_tarjan(V, edges):
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)
    
    disc = [-1] * V
    low = [-1] * V
    visited = [False] * V
    parent = [-1] * V
    ap = [False] * V
    
    time = 0
    
    def dfs(u):
        nonlocal time
        visited[u] = True
        disc[u] = low[u] = time
        time += 1
        
        children = 0
        
        for v in graph[u]:
            if not visited[v]:
                parent[v] = u
                children += 1
                dfs(v)
                
                low[u] = min(low[u], low[v])
                
                # Condition 1: root with 2+ children
                if parent[u] == -1 and children > 1:
                    ap[u] = True
                
                # Condition 2: low[child] >= disc[u]
                if parent[u] != -1 and low[v] >= disc[u]:
                    ap[u] = True
            
            elif v != parent[u]:  # Back edge
                low[u] = min(low[u], disc[v])
    
    for i in range(V):
        if not visited[i]:
            dfs(i)
    
    return [i for i in range(V) if ap[i]]


# Example test
V = 5
edges = [(0,1), (1,2), (2,0), (1,3), (3,4)]
print("Optimized Tarjan APs:", articulation_points_tarjan(V, edges))


Optimized Tarjan APs: [1, 3]


### Alternative Approaches

1. BFS-based articulation detection (less common)
- Traditionally, articulation points are computed using DFS.
- BFS-based methods exist but are complex and not typically used.

2. Using edge-biconnected components (bridges + articulation points)
- Using DSU (Union-Find) + bridge-finding can also find articulation points indirectly.

## Complexity Analysis

| Approach               | Time Complexity  | Space Complexity |
| ---------------------- | ---------------- | ---------------- |
| **Brute Force**        | `O(V * (V + E))` | `O(V + E)`       |
| **Tarjan (Optimized)** | `O(V + E)`       | `O(V)`           |


#### Thank You!!